# First, import the packages and the programs from the .py file

In [ ]:
import io
import os
import cv2
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.cloud import vision
import json
from PIL import Image, ImageDraw
from textractor import Textractor
from textractor.visualizers.entitylist import EntityList
from textractor.data.constants import TextractFeatures, Direction, DirectionalFinderType
import Process_Programs as pp


Define the folders you will be working in

In [ ]:
input_folder = "images/"

output_folder = "output/"

# Single image processing

In [ ]:
filename = "1888_Page_161.png"
image_path = os.path.join(input_folder, filename)
full_image = cv2.imread(image_path)


#Put modifications to default settings ABOVE here
#Suppose you want to change the side margins to be 8% of the page. 
#Then type: pp.full['left_margin_percent'] = 8
mod_image = pp.draw_margins_splits(full_image, **pp.full);


#If you accept the parameters, save the single image in the output folder
output_path = os.path.join(output_folder, 'processed_' + filename)

try:
    cv2.imwrite(output_path, mod_image)
except:
    pass

# Split image processing

In [ ]:
split_filename = "126.png"
image_path = os.path.join(input_folder, split_filename)
full_image = cv2.imread(image_path)


#Put modifications to default settings ABOVE here
#Suppose you want to change the side margins to be 8% of the page. 
#Then type: pp.full['left_margin_percent'] = 8
left_image, right_image = pp.draw_margins_splits(full_image, **pp.vsplit);


#If you accept the parameters, save the split images in the output folder
L_output_path = os.path.join(output_folder, 'processed_L_' + split_filename)
R_output_path = os.path.join(output_folder, 'processed_R_' + split_filename)

try:
    cv2.imwrite(L_output_path, left_image)
    cv2.imwrite(R_output_path, right_image)
except:
    pass

# Send an example image through Textract and/or Cloud Vision

Initialize Textractor client, modify region if required

In [ ]:
extractor = Textractor(profile_name="default")

The single US Government image with a single table

In [ ]:
# Load the document image with Pillow. This is necessary for Textract to run.
image = Image.open(output_path)

# Analyze the document and specify you want to extract tables
document = extractor.analyze_document(
    file_source=image,
    features=[TextractFeatures.TABLES],
    save_image=True
)

#Show the summary statistics of the detected objects

document

In [ ]:
#Create a variable for the detected words
words_entity_list = document.words
#Create a variable for the detected tables
table_list = document.tables

#Load the document image with cv2. This is necessary to draw the bounding boxes
sing_image = cv2.imread(output_path)
#Draw the bounding boxes in Textract
pp.textract_boxes(sing_image, words_entity_list, table_list, show_image=True)

Initialize Cloud Vision

In [ ]:
#Authenticate Google Cloud

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'ServiceAccountToken.json'
client = vision.ImageAnnotatorClient()

Run the same image through Cloud Vision

In [ ]:
#Define where you want to output the json file
json_output = os.path.join(output_folder, "1888_Page_161.json")

#Run Google Cloud Vision on the single image

pp.gcloud_boxes(sing_image, json_output, show_image=True);

# For another example, use one of the split images from above

In [ ]:
# Load the document image with Pillow. This is necessary for Textract to run.
image = Image.open(L_output_path)

# Analyze the document and specify you want to extract tables
document = extractor.analyze_document(
    file_source=image,
    features=[TextractFeatures.TABLES],
    save_image=True
)

#Show the summary statistics of the detected objects

document

In [ ]:
#Create a variable for the detected words
words_entity_list = document.words
#Create a variable for the detected tables
table_list = document.tables

#Load the document image with cv2. This is necessary to draw the bounding boxes
L_image = cv2.imread(L_output_path)

pp.textract_boxes(L_image, words_entity_list, table_list, show_image=True)

Run the same image through Cloud Vision

In [ ]:
#Define where you want to output the json file
json_output = os.path.join(output_folder, "L_126.json")

#Run Google Cloud Vision on the single image

pp.gcloud_boxes(L_image, json_output, show_image=True);

# There are other features you can exploit as well. See below.

Saving the output of your first detected table to an excel file.

In [ ]:
#document.tables[0].to_excel("output.xlsx")

Bypassing the textractor package, obtaining the bounding boxes and raw .json output

In [ ]:
#json_output = 'Textractoutput.json'


#pp.Textract_process_image(output_path, json_output)